In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import math
#from dateutil import parser
#import os, gc

import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import plotly.express as px
import seaborn as sns
mpl.style.use('seaborn') # так красивее
%pylab inline
%matplotlib inline

import json
import requests
from datetime import date, datetime, timedelta

from aiogram import Bot, types
from aiogram.dispatcher import Dispatcher, FSMContext
from aiogram.utils import executor
#import telebot

import nest_asyncio
nest_asyncio.apply()
from aiogram.dispatcher import filters
from aiogram.dispatcher.filters.state import State, StatesGroup
from html import *
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton, ReplyKeyboardRemove, InlineKeyboardMarkup
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from openpyxl import Workbook

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% ! important; }<style>"))

import dataframe_image as dfi
import asyncio

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:
user_id = str(503499212)
user_api = '...'

In [30]:
#towrite = io.BytesIO()
#df_styled.to_excel(towrite)  # write to BytesIO buffer
#towrite.seek(0) 
#file_like = BufferedReader(BytesIO(towrite.getvalue()))

In [4]:
df_user_api_excel = pd.read_excel("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\df_user_api_excel.xlsx").astype(str)
df_user_api_excel

,user_id,user_api
0,503499212,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc...
1,388842976,Dhdjdjcjdjdk


In [60]:
sum(orders[orders['yearweek_num']==202301]['PriceWithDiscount'])

149104.68050000002

In [88]:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(503499212)].reset_index(drop=True)["user_api"].iloc[-1]}'})
       # while response_orders.status_code == 429:
           # await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
           # await asyncio.sleep(30)
            #response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
             #   headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        orders = orders_by_article(response_orders)
        orders = orders[(orders['date'] >= str(current_date)) & (orders['cancel_dt'] == '0001-01-01T00:00:00')]
        pivot_orders = pivot_ord(orders)
        response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(503499212)].reset_index(drop=True)["user_api"].iloc[-1]}'})
       # while response_stocks.status_code == 429:
          #  await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
           # await asyncio.sleep(30)
           # response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            #    headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        pivot_orders = pivot_ord_stock(pivot_orders, response_stocks)
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            print(f"Заказы за сегодня, общая сумма -" 
                f" {int(round(sum(pivot_orders['Сумма_в_рублях']), 0))} (с учётом СПП),"
                f" общее количество - {sum(pivot_orders['Количество_в_шт'])}.\nКартинкой выводится до топ 50 артикулов по сумме заказов.")
           

Заказы за сегодня, общая сумма - 1946 (с учётом СПП), общее количество - 2.
Картинкой выводится до топ 50 артикулов по сумме заказов.


In [99]:
msg_dict = {'Заказы_за_сегодня': 'Сегодня', 'Заказы_за_сегодня_Excel': 'Сегодня', 1: 'mango', 2: 'pawpaw'}
dict_sample['Заказы_за_сегодня']

'Сегодня'

In [ ]:
    keyboard = InlineKeyboardMarkup(row_width=2)
    keyboard.row(types.InlineKeyboardButton(text="Заказы за сегодня", callback_data="Заказы_за_сегодня"), 
                 types.InlineKeyboardButton(text="Заказы за сегодня с Excel", callback_data="Заказы_за_сегодня_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за вчера", callback_data="Заказы_за_вчера"), 
                 types.InlineKeyboardButton(text="Заказы за вчера с Excel", callback_data="Заказы_за_вчера_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за эту неделю", callback_data="Заказы_за_эту_неделю"), 
                 types.InlineKeyboardButton(text="Заказы за эту неделю с Excel", callback_data="Заказы_за_эту_неделю_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за прошлую неделю", callback_data="Заказы_за_прошлую_неделю"), 
                 types.InlineKeyboardButton(text="Заказы за прошлую с Excel", callback_data="Заказы_за_прошлую_неделю_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за этот месяц", callback_data="Заказы_за_этот_месяц"), 
                 types.InlineKeyboardButton(text="Заказы за этот месяц с Excel", callback_data="Заказы_за_этот_месяц_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за прошлый месяц", callback_data="Заказы_за_прошлый_месяц"), 
                 types.InlineKeyboardButton(text="Заказы за прошлый месяц с Excel", callback_data="Заказы_за_прошлый_месяц_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за последние 30 дней", callback_data="Заказы_за_последние_30_дней"), 
                 types.InlineKeyboardButton(text="Заказы за последние 30 дней с Excel", callback_data="Заказы_за_последние_30_дней_Excel"))
    await message.answer("Выберите период, заказы по которому вы хотите узнать", reply_markup=keyboard)



msg_dict = {'Заказы_за_сегодня': 'сегодня', 'Заказы_за_сегодня_Excel': 'сегодня', 'Заказы_за_вчера': 'вчера', 'Заказы_за_вчера_Excel': 'вчера', 'Заказы_за_эту_неделю': 'эту неделю', 'Заказы_за_эту_неделю': 'эту неделю',
           'Заказы_за_прошлую_неделю': 'прошлую неделю', 'Заказы_за_прошлую_неделю': 'прошлую неделю', 'Заказы_за_этот_месяц': 'этот месяц', 'Заказы_за_этот_месяц': 'этот месяц', 
           'Заказы_за_прошлый_месяц': 'прошлый месяц', 'Заказы_за_прошлый_месяц': 'прошлый месяц', 'Заказы_за_последние_30_дней': 'последние 30 дней', 'Заказы_за_последние_30_дней': 'последние 30 дней'}

In [ ]:
tg_bot_token = '...'
#wb_api = '...'
storage = MemoryStorage()
bot = Bot(token=tg_bot_token)
dp = Dispatcher(bot, storage=storage)

class FSMdata(StatesGroup):
    wb_api = State()
    report_date = State()

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
# создаём клавиатуру
kb_1 = KeyboardButton('/Ввести_API_ключ\U0001F5A5')
kb_2 = KeyboardButton('/Заказы_по_товарам')
kb_3 = KeyboardButton('/Продажи_по_товарам')
kb_4 = KeyboardButton('/График_заказов_и_продаж')
kb_5 = KeyboardButton('/Остатки_по_товарам')
kb_6 = KeyboardButton('/Заказы_по_товарам_выбрать_даты')
kb_7 = KeyboardButton('/Продажи_по_товарам_выбрать_даты')

kb_api = ReplyKeyboardMarkup(resize_keyboard=True)
kb_api.add(kb_1)
kb_main = ReplyKeyboardMarkup(resize_keyboard=True)
kb_main.row(kb_2, kb_3).row(kb_6, kb_7).row(kb_4, kb_5)

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
msg_dict = {'Заказы_за_сегодня': 'Сегодня', 'Заказы_за_сегодня_Excel': 'Сегодня', 1: 'mango', 2: 'pawpaw'}

def orders_by_article(response_orders):
    cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
    orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
    orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
    orders['week_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.isocalendar().week
    orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
    orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
    orders['week_num'] = orders['week_num'].astype(str).apply(lambda x: x.zfill(2))
    orders['month_num'] = orders['month_num'].astype(str).apply(lambda x: x.zfill(2))
    orders['yearweek_num'] = (orders['year_num'].astype(str) + orders['week_num'].astype(str)).astype(int)
    orders['yearmonth_num'] = (orders['year_num'].astype(str) + orders['month_num'].astype(str)).astype(int)
    orders.loc[(orders['week_num'].astype(int) >= 50) & (orders['month_num'].astype(int) == 1), 'yearweek_num'] -= 100
    return orders

def pivot_ord(orders):
    return pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['supplierArticle'], 
    aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders, 
    values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
    left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index()

def pivot_ord_stock(pivot_orders, response_stocks):
    stocks = pd.DataFrame(json.loads(response_stocks.content.decode('utf-8')))[['supplierArticle', 'quantity']].groupby(['supplierArticle']).sum().reset_index().rename(columns={'quantity': 'Остатки'})
    pivot_orders = pd.merge(pivot_orders, stocks, how="left", left_on='supplierArticle', right_on='supplierArticle').rename(columns={'qty': 'Количество_в_шт', 
                'PriceWithDiscount': 'Сумма_в_рублях', 'supplierArticle': 'Артикул_поставщика'})
    df_styled = pivot_orders[['Артикул_поставщика', 'Количество_в_шт', 'Сумма_в_рублях', 'Остатки']].head(50).style.set_precision(0).background_gradient(axis=0, gmap=pivot_orders['Сумма_в_рублях'], cmap='BuPu')
    dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
    return pivot_orders
    
def excel_saver(pivot_orders):
    writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
    pivot_orders.to_excel(writer, sheet_name='Sheet1', index=False)
    writer.save()

def ord_req(user_id, df_user_api_excel):
    return requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0', headers={'Authorization': f'{user_id}'})

def stock_req(user_id, df_user_api_excel):
    return requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0', headers={'Authorization': f'{user_id}'})

# Достаём API продавца
current_date = date.today()
df_user_api_excel = pd.read_excel("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\df_user_api_excel.xlsx").astype(str)

@dp.message_handler(commands=['Ввести_API_ключ', 'start', 'api', 'Привет', 'привет'], state=None)
async def ask_API(message: types.Message):
    
    user_full_name = message.from_user.full_name
    user_id = message.from_user.id
    shape = df_user_api_excel[df_user_api_excel["user_id"] == str(user_id)]["user_api"].shape[0]
    
    if df_user_api_excel['user_id'].str.contains(str(message.from_user.id)).any():
        if requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization':
            f'{df_user_api_excel[df_user_api_excel["user_id"] == str(user_id)]["user_api"].reset_index(drop=True).iloc[-1]}'}).status_code == 200:
            
            await bot.send_message(message.chat.id, f'Привет, ты уже авторизирован и можешь запрашивать статистику!',
                              reply_markup=kb_main)
        else: 
            await bot.send_message(message.chat.id, f'Привет, вижу, что ты уже пытался пройти авторизацию, но ввёл неверный '
                f'API ключ \U0001F614. Попробуй ввести его ещё раз, нажав на кнопку /Ввести_API_ключ\U0001F5A5, которая'
                f' находится внизу. \nAPI, который ты вводил до этого: '
                f'{df_user_api_excel[df_user_api_excel["user_id"] == str(user_id)].reset_index(drop=True)["user_api"].iloc[-1]}',
                              reply_markup=kb_api)
        #await message.delete_reply_markup()
    else:
        await FSMdata.wb_api.set()

        await bot.send_message(message.chat.id, f'Привет, {user_full_name}, это бот статистики заказов и выкупов Wildberries!\n'
        f'Введи сюда свой API ключ, который находится в настройках профиля на портале.\n'
        f'Вот ссылка: https://seller.wildberries.ru/supplier-settings/access-to-new-api\n'
        f'Здесь нажми "Создать новый ключ", выбери "Статистика" (так мы будем иметь доступ только к статистике, но не иметь доступа к созданию и редактированию карточек товара и т. д.)\n'
        f'Теперь скопируй этот большой набор символов и отправь мне. Жду!', reply_markup=kb_api)
        #await message.answer(f'{user_full_name}, отправь в чат API ключ', reply_markup=kb_api)
    
@dp.message_handler(state=FSMdata.wb_api)
async def get_API(message: types.Message, state: FSMContext):
    global df_user_api_excel
    user_id = message.from_user.id
    async with state.proxy() as data:
        data['wb_api'] = message.text

    async with state.proxy() as data:
        user_id = str(message.from_user.id)
        user_api = "'".join(str(data)[61:].split("'")[:-1])
        if requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization':
            f'{user_api}'}).status_code != 200:
            await bot.send_message(message.chat.id, f'Мне кажется, что это не API ключ \U0001FAE0 \nПопробуй ещё раз. Следуй строго по инструкциям из первого сообщения.', 
                                   reply_markup=kb_api)
        else:
        #async with state.proxy() as data:
            #data['wb_api'] = message.text

        #async with state.proxy() as data:
            #user_id = str(message.from_user.id)
            #user_api = "'".join(str(data)[61:].split("'")[:-1])
            df_user_api_excel.loc[len(df_user_api_excel)] = [user_id, user_api]
            writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\df_user_api_excel.xlsx' , engine='xlsxwriter')
            df_user_api_excel.drop_duplicates(subset=['user_id'], keep='last').to_excel(writer, sheet_name='Sheet1', index=False)
            writer.save()
            df_user_api_excel = pd.read_excel("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\df_user_api_excel.xlsx").astype(str)
            df_user_api_excel[df_user_api_excel["user_id"] == str(user_id)].reset_index(drop=True)["user_api"].iloc[-1]
            await bot.send_message(message.chat.id, f'Твой ключ API: {df_user_api_excel[df_user_api_excel["user_id"] == str(user_id)].reset_index(drop=True)["user_api"].iloc[-1]}\n'
                                   f'Теперь давай опробуем наш функционал.',
                                   reply_markup=kb_main)
    await state.finish()

        

        
@dp.message_handler(commands=['Заказы_по_товарам_выбрать_даты', 'Продажи_по_товарам_выбрать_даты'], state=None)
async def report_dates(message: types.Message):        
        await FSMdata.report_date.set()
        await bot.send_message(message.chat.id, f'Введи даты в СТРОГО ЗАДАННОМ ФОРМАТЕ "Заказы dd.mm.yyyy-dd.mm.yyyy" или "Продажи dd.mm.yyyy-dd.mm.yyyy"\n'
                               f'Примеры: Заказы 01.12.2022-28.12.2022\nПродажи 11.11.2021-25.11.2022', reply_markup=kb_main)
        
      
        
###################################################################   ОСТАТКИ     #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!    
###################################################################   ОСТАТКИ     #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!        
###################################################################   ОСТАТКИ     #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!       
                
@dp.message_handler(commands=['Остатки_по_товарам'], state=None) 
async def def_stocks(message: types.Message):
    try:
        response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(message.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        stocks = pd.DataFrame(json.loads(response_stocks.content.decode('utf-8')))[['supplierArticle', 'quantity']].groupby(['supplierArticle']).sum().reset_index().rename(columns={
            'quantity': 'Остатки', 'supplierArticle': 'Артикул_поставщика'}).sort_values(by=['Остатки'], ascending=True, na_position='first').reset_index(drop=True)
        df_styled = stocks[stocks['Остатки'] > 0][['Артикул_поставщика', 'Остатки']].head(50).style.set_precision(0).background_gradient(axis=0, gmap=stocks['Остатки'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await message.message.answer_photo(photo = photo, caption=f"Количество товара на складах в разбивке по артикулу поставщика, на картинке показываются до топ 50 товаров," 
                                            f"в которых минимальный остаток, не больше 0 (в Excel выводится вся информация).\nВсего у вас {sum(stocks['Остатки'])} шт. товара (в совокупности) на данный момент.")       
        writer = pd.ExcelWriter('D:\\грузи сюда\\Всё по WB\\ВСЁ НАШЕЕЕЕЕЕ\\stocks.xlsx' , engine='xlsxwriter')
        stocks.to_excel(writer, sheet_name='Sheet1', index=False)
        writer.save()
        with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\stocks.xlsx", 'rb') as file:
            await message.message.answer_document(file)
    except:
        await message.message.answer(f'\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')     
        
        
##########   З   А   К   А   З   Ы      З   А   #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!    
##########   З   А   К   А   З   Ы      З   А   #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!        
##########   З   А   К   А   З   Ы      З   А   #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!      

    
@dp.message_handler(commands="Заказы_по_товарам")
async def InlKB(message: types.Message):
    keyboard = InlineKeyboardMarkup(row_width=2)
    keyboard.row(types.InlineKeyboardButton(text="Заказы за сегодня", callback_data="Заказы_за_сегодня"), 
                 types.InlineKeyboardButton(text="Заказы за сегодня с Excel", callback_data="Заказы_за_сегодня_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за вчера", callback_data="Заказы_за_вчера"), 
                 types.InlineKeyboardButton(text="Заказы за вчера с Excel", callback_data="Заказы_за_вчера_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за эту неделю", callback_data="Заказы_за_эту_неделю"), 
                 types.InlineKeyboardButton(text="Заказы за эту неделю с Excel", callback_data="Заказы_за_эту_неделю_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за прошлую неделю", callback_data="Заказы_за_прошлую_неделю"), 
                 types.InlineKeyboardButton(text="Заказы за прошлую с Excel", callback_data="Заказы_за_прошлую_неделю_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за этот месяц", callback_data="Заказы_за_этот_месяц"), 
                 types.InlineKeyboardButton(text="Заказы за этот месяц с Excel", callback_data="Заказы_за_этот_месяц_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за прошлый месяц", callback_data="Заказы_за_прошлый_месяц"), 
                 types.InlineKeyboardButton(text="Заказы за прошлый месяц с Excel", callback_data="Заказы_за_прошлый_месяц_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Заказы за последние 30 дней", callback_data="Заказы_за_последние_30_дней"), 
                 types.InlineKeyboardButton(text="Заказы за последние 30 дней с Excel", callback_data="Заказы_за_последние_30_дней_Excel"))
    await message.answer("Выберите период, заказы по которому вы хотите узнать", reply_markup=keyboard) 
     

@dp.callback_query_handler(text=["Заказы_за_сегодня", "Заказы_за_сегодня_Excel"])
async def orders(call: types.CallbackQuery): 
    try:
        user_id = df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]
        response_orders = ord_req(user_id, df_user_api_excel) ###
        while response_orders.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_orders = ord_req(user_id, df_user_api_excel) ###
        orders = orders_by_article(response_orders) ###
        if (call.data == 'Заказы_за_сегодня') | (call.data == 'Заказы_за_сегодня_Excel'):
            orders = orders[(orders['date'] >= str(current_date)) & (orders['cancel_dt'] == '0001-01-01T00:00:00')]
        elif (call.data == 'Заказы_за_вчера') | (call.data == 'Заказы_за_вчера_Excel'):
            orders = orders[(orders['date'] < str(current_date)) & (orders['date'] >= str(current_date - timedelta(1))) & (orders['cancel_dt'] == '0001-01-01T00:00:00')]
        pivot_orders = pivot_ord(orders) ###
        response_stocks = stock_req(user_id, df_user_api_excel) ###
        while response_stocks.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_stocks = stock_req(user_id, df_user_api_excel) ###
        pivot_orders = pivot_ord_stock(pivot_orders, response_stocks) ###
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказы за {msg_dict[call.data]}, общая сумма -" 
                                            f" {int(round(sum(pivot_orders['Сумма_в_рублях']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_orders['Количество_в_шт'])}.\nКартинкой выводится до топ 50 артикулов по сумме заказов.")
            if call.data == 'Заказы_за_сегодня':
                await call.answer()
            elif call.data == 'Заказы_за_сегодня_Excel':
                excel_saver(pivot_orders) ###
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        try:
            if orders.shape[0] == 0:
                await call.message.answer(f'За этот период ещё не было заказов \U0001F9D0\U0001F9D0\U0001F9D0')
                await call.answer()
        except:
            await call.message.answer(f'\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
    
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    

@dp.callback_query_handler(text=["Заказы_за_вчера", 'Заказы_за_вчера_Excel'])
async def orders(call: types.CallbackQuery): 
    try:
        user_id = df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]
        response_orders = ord_req(user_id, df_user_api_excel) ###
        while response_orders.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_orders = ord_req(user_id, df_user_api_excel) ###
        orders = orders_by_article(response_orders) ###
        orders = orders[(orders['date'] < str(current_date)) & (orders['date'] >= str(current_date - timedelta(1))) & (orders['cancel_dt'] == '0001-01-01T00:00:00')]
        pivot_orders = pivot_ord(orders) ###
        response_stocks = stock_req(user_id, df_user_api_excel) ###
        while response_stocks.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_stocks = stock_req(user_id, df_user_api_excel) ###
        pivot_orders = pivot_ord_stock(pivot_orders, response_stocks) ###
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказы за вчера, общая сумма -" 
                                            f" {int(round(sum(pivot_orders['Сумма_в_рублях']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_orders['Количество_в_шт'])}.\nКартинкой выводится до топ 50 артикулов по сумме заказов.")
            if call.data == 'Заказы_за_вчера':
                await call.answer()
            elif call.data == 'Заказы_за_вчера_Excel':
                excel_saver(pivot_orders) ###
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        try:
            if orders.shape[0] == 0:
                await call.message.answer(f'За этот период ещё не было заказов \U0001F9D0\U0001F9D0\U0001F9D0')
                await call.answer()
        except:
            await call.message.answer(f'\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
    
@dp.callback_query_handler(text=["Заказы_за_эту_неделю", 'Заказы_за_эту_неделю_Excel'])
async def orders(call: types.CallbackQuery):
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        while response_orders.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
                headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders['week_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.isocalendar().week
        orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
        orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
        orders['week_num'] = orders['week_num'].astype(str).apply(lambda x: x.zfill(2))
        orders['yearweek_num'] = (orders['year_num'].astype(str) + orders['week_num'].astype(str)).astype(int)
        orders.loc[(orders['week_num'].astype(int) >= 50) & (orders['month_num'].astype(int) == 1), 'yearweek_num'] -= 100
        orders = orders[(orders['yearweek_num'] >= max(orders['yearweek_num'])) & (orders['cancel_dt'] == '0001-01-01T00:00:00')]
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['supplierArticle'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders, 
            values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
            left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index()
        response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        while response_stocks.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
                headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        stocks = pd.DataFrame(json.loads(response_stocks.content.decode('utf-8')))[['supplierArticle', 'quantity']].groupby(['supplierArticle']).sum().reset_index().rename(columns={'quantity': 'Остатки'})
        pivot_orders = pd.merge(pivot_orders, stocks, how="left", left_on='supplierArticle', right_on='supplierArticle').rename(columns={'qty': 'Количество_в_шт', 
                    'PriceWithDiscount': 'Сумма_в_рублях', 'supplierArticle': 'Артикул_поставщика'})  
        df_styled = pivot_orders[['Артикул_поставщика', 'Количество_в_шт', 'Сумма_в_рублях', 'Остатки']].head(50).style.set_precision(0).background_gradient(axis=0, gmap=pivot_orders['Сумма_в_рублях'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказы за эту неделю, общая сумма -" 
                                            f" {int(round(sum(pivot_orders['Сумма_в_рублях']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_orders['Количество_в_шт'])}.\nКартинкой выводится до топ 50 артикулов по сумме заказов.") 
            if call.data == 'Заказы_за_эту_неделю':
                await call.answer()
            elif call.data == 'Заказы_за_эту_неделю_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_orders.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if orders.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer(f'\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------         
        
@dp.callback_query_handler(text=["Заказы_за_прошлую_неделю", 'Заказы_за_прошлую_неделю_Excel'])
async def orders(call: types.CallbackQuery):
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        while response_orders.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
                headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders['week_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.isocalendar().week
        orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
        orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
        orders['week_num'] = orders['week_num'].astype(str).apply(lambda x: x.zfill(2))
        orders['yearweek_num'] = (orders['year_num'].astype(str) + orders['week_num'].astype(str)).astype(int)
        orders.loc[(orders['week_num'] >= 50) & (orders['month_num'] == 1), 'yearweek_num'] -= 100
        orders = orders[(orders['yearweek_num'] == np.sort(orders['yearweek_num'].unique().astype(int))[-2]) & (orders['cancel_dt'] == '0001-01-01T00:00:00')] 
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['supplierArticle'],
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders,
            values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
            left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index()
        response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        while response_stocks.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
                headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        stocks = pd.DataFrame(json.loads(response_stocks.content.decode('utf-8')))[['supplierArticle', 'quantity']].groupby(['supplierArticle']).sum().reset_index().rename(columns={'quantity': 'Остатки'})
        pivot_orders = pd.merge(pivot_orders, stocks, how="left", left_on='supplierArticle', right_on='supplierArticle').rename(columns={'qty': 'Количество_в_шт', 
                    'PriceWithDiscount': 'Сумма_в_рублях', 'supplierArticle': 'Артикул_поставщика'})       
        df_styled = pivot_orders[['Артикул_поставщика', 'Количество_в_шт', 'Сумма_в_рублях', 'Остатки']].head(50).style.set_precision(0).background_gradient(axis=0, gmap=pivot_orders['Сумма_в_рублях'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказы за прошлую неделю, общая сумма -" 
                                            f" {int(round(sum(pivot_orders['Сумма_в_рублях']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_orders['Количество_в_шт'])}.\nКартинкой выводится до топ 50 артикулов по сумме заказов.") 
            if call.data == 'Заказы_за_прошлую_неделю':
                await call.answer()
            elif call.data == 'Заказы_за_прошлую_неделю_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_orders.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if orders.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
        
@dp.callback_query_handler(text=["Заказы_за_этот_месяц", 'Заказы_за_этот_месяц_Excel'])
async def orders(call: types.CallbackQuery):
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        while response_orders.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
                headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
        orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
        orders['month_num'] = orders['month_num'].astype(str).apply(lambda x: x.zfill(2))
        orders['yearmonth_num'] = (orders['year_num'].astype(str) + orders['month_num'].astype(str)).astype(int)
        orders = orders[(orders['yearmonth_num'] >= max(orders['yearmonth_num'])) & (orders['cancel_dt'] == '0001-01-01T00:00:00')]
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', 
            index=['supplierArticle'], aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), 
            pd.pivot_table(orders, values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', 
            margins=False), how="left", left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index()
        response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        while response_stocks.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
                headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        stocks = pd.DataFrame(json.loads(response_stocks.content.decode('utf-8')))[['supplierArticle', 'quantity']].groupby(['supplierArticle']).sum().reset_index().rename(columns={'quantity': 'Остатки'})
        pivot_orders = pd.merge(pivot_orders, stocks, how="left", left_on='supplierArticle', right_on='supplierArticle').rename(columns={'qty': 'Количество_в_шт', 
                    'PriceWithDiscount': 'Сумма_в_рублях', 'supplierArticle': 'Артикул_поставщика'})
        df_styled = pivot_orders[['Артикул_поставщика', 'Количество_в_шт', 'Сумма_в_рублях', 'Остатки']].head(50).style.set_precision(0).background_gradient(axis=0, gmap=pivot_orders['Сумма_в_рублях'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказы за этот месяц, общая сумма -" 
                                            f" {int(round(sum(pivot_orders['Сумма_в_рублях']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_orders['Количество_в_шт'])}.\nКартинкой выводится до топ 50 артикулов по сумме заказов.") 
            if call.data == 'Заказы_за_этот_месяц':
                await call.answer()
            elif call.data == 'Заказы_за_этот_месяц_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_orders.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if orders.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru') 
            await call.answer()

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
        
@dp.callback_query_handler(text=["Заказы_за_прошлый_месяц", 'Заказы_за_прошлый_месяц_Excel'])
async def orders(call: types.CallbackQuery):
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        while response_orders.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
                headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
        orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
        orders['month_num'] = orders['month_num'].astype(str).apply(lambda x: x.zfill(2))
        orders['yearmonth_num'] = (orders['year_num'].astype(str) + orders['month_num'].astype(str)).astype(int)
        orders = orders[(orders['yearmonth_num'] == np.sort(orders['yearmonth_num'].unique().astype(int))[-2]) & (orders['cancel_dt'] == '0001-01-01T00:00:00')]
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['supplierArticle'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders, 
            values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
            left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index()
        response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        while response_stocks.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
                headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        stocks = pd.DataFrame(json.loads(response_stocks.content.decode('utf-8')))[['supplierArticle', 'quantity']].groupby(['supplierArticle']).sum().reset_index().rename(columns={'quantity': 'Остатки'})
        pivot_orders = pd.merge(pivot_orders, stocks, how="left", left_on='supplierArticle', right_on='supplierArticle').rename(columns={'qty': 'Количество_в_шт', 
                    'PriceWithDiscount': 'Сумма_в_рублях', 'supplierArticle': 'Артикул_поставщика'})
        df_styled = pivot_orders[['Артикул_поставщика', 'Количество_в_шт', 'Сумма_в_рублях', 'Остатки']].head(50).style.set_precision(0).background_gradient(axis=0, gmap=pivot_orders['Сумма_в_рублях'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказы за прошлый месяц, общая сумма -" 
                                            f" {int(round(sum(pivot_orders['Сумма_в_рублях']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_orders['Количество_в_шт'])}.\nКартинкой выводится до топ 50 артикулов по сумме заказов.") 
            if call.data == 'Заказы_за_прошлый_месяц':
                await call.answer()
            elif call.data == 'Заказы_за_прошлый_месяц_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_orders.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if orders.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()

#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
        
@dp.callback_query_handler(text=["Заказы_за_последние_30_дней", 'Заказы_за_последние_30_дней_Excel'])
async def orders(call: types.CallbackQuery):
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        while response_orders.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
                headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders = orders[(orders['date'] >= str(current_date - timedelta(30))) & (orders['cancel_dt'] == '0001-01-01T00:00:00')]
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['supplierArticle'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders, 
            values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
            left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index()
        response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        while response_stocks.status_code == 429:
            await call.message.answer(f'Сейчас много запросов, придётся подождать \U0001F9D0\U0001F9D0\U0001F9D0\nОжидание продлится не больше минуты')
            await asyncio.sleep(30)
            response_stocks = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/stocks?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
                headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        stocks = pd.DataFrame(json.loads(response_stocks.content.decode('utf-8')))[['supplierArticle', 'quantity']].groupby(['supplierArticle']).sum().reset_index().rename(columns={'quantity': 'Остатки'})
        pivot_orders = pd.merge(pivot_orders, stocks, how="left", left_on='supplierArticle', right_on='supplierArticle').rename(columns={'qty': 'Количество_в_шт', 
                    'PriceWithDiscount': 'Сумма_в_рублях', 'supplierArticle': 'Артикул_поставщика'})
        df_styled = pivot_orders[['Артикул_поставщика', 'Количество_в_шт', 'Сумма_в_рублях', 'Остатки']].head(50).style.set_precision(0).background_gradient(axis=0, gmap=pivot_orders['Сумма_в_рублях'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказы за последние 30 дней, общая сумма -" 
                                            f" {int(round(sum(pivot_orders['Сумма_в_рублях']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_orders['Количество_в_шт'])}.\nКартинкой выводится до топ 50 артикулов по сумме заказов.")
            if call.data == 'Заказы_за_последние_30_дней':
                await call.answer()
            elif call.data == 'Заказы_за_последние_30_дней_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_orders.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if orders.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
        
        
##########   П   Р   О   Д   А   Ж   И      З   А   #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!         
##########   П   Р   О   Д   А   Ж   И      З   А   #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!         
##########   П   Р   О   Д   А   Ж   И      З   А   #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  
        
@dp.message_handler(commands="Продажи_по_товарам")
async def InlKB(message: types.Message):
    keyboard = InlineKeyboardMarkup(row_width=2)
    keyboard.row(types.InlineKeyboardButton(text="Продажи за сегодня", callback_data="Продажи_за_сегодня"), 
                 types.InlineKeyboardButton(text="Продажи за сегодня с Excel", callback_data="Продажи_за_сегодня_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Продажи за вчера", callback_data="Продажи_за_вчера"), 
                 types.InlineKeyboardButton(text="Продажи за вчера с Excel", callback_data="Продажи_за_вчера_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Продажи за эту неделю", callback_data="Продажи_за_эту_неделю"), 
                 types.InlineKeyboardButton(text="Продажи за эту неделю с Excel", callback_data="Продажи_за_эту_неделю_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Продажи за прошлую неделю", callback_data="Продажи_за_прошлую_неделю"), 
                 types.InlineKeyboardButton(text="Продажи за прошлую с Excel", callback_data="Продажи_за_прошлую_неделю_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Продажи за этот месяц", callback_data="Продажи_за_этот_месяц"), 
                 types.InlineKeyboardButton(text="Продажи за этот месяц с Excel", callback_data="Продажи_за_этот_месяц_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Продажи за прошлый месяц", callback_data="Продажи_за_прошлый_месяц"), 
                 types.InlineKeyboardButton(text="Продажи за прошлый месяц с Excel", callback_data="Продажи_за_прошлый_месяц_Excel"))
    keyboard.row(types.InlineKeyboardButton(text="Продажи за последние 30 дней", callback_data="Продажи_за_последние_30_дней"), 
                 types.InlineKeyboardButton(text="Продажи за последние 30 дней с Excel", callback_data="Продажи_за_последние_30_дней_Excel"))
    await message.answer("Выберите период, продажи по которому вы хотите узнать", reply_markup=keyboard)      
        
        
@dp.callback_query_handler(text=["Продажи_за_сегодня", 'Продажи_за_сегодня_Excel'])
async def sales(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
                'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales['PriceWithDiscount'] = sales['totalPrice']*((100-sales['discountPercent'])/100)*((100-sales['promoCodeDiscount'])/100)*((100-sales['spp'])/100)
        sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        sales = sales[(sales['date'] >= str(current_date)) & (sales['IsStorno'] == 0) & (sales['PriceWithDiscount'] > 0)]
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['supplierArticle'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
            left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index().sort_values(by=['PriceWithDiscount'], 
                                                            ascending=False, na_position='first').reset_index(drop=True)
        df_styled = pivot_sales[['supplierArticle', 'qty', 'PriceWithDiscount']].head(50).style.set_precision(0).background_gradient(
            axis=0, gmap=pivot_sales['PriceWithDiscount'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Продажи за сегодня, общая сумма -" 
                                            f" {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_sales['qty'])}.\nКартинкой выводится до топ 50 артикулов по сумме продаж.")
            if call.data == 'Продажи_за_сегодня':
                await call.answer()
            elif call.data == 'Продажи_за_сегодня_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_sales.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
        
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------        

@dp.callback_query_handler(text=["Продажи_за_вчера", 'Продажи_за_вчера_Excel'])
async def sales(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
                'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales['PriceWithDiscount'] = sales['totalPrice']*((100-sales['discountPercent'])/100)*((100-sales['promoCodeDiscount'])/100)*((100-sales['spp'])/100)
        sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        sales = sales[(sales['date'] < str(current_date)) & (sales['date'] >= str(current_date - timedelta(1))) 
                                & (sales['IsStorno'] == 0) & (sales['PriceWithDiscount'] > 0)]
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['supplierArticle'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
            left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index().sort_values(by=['PriceWithDiscount'], 
                                                            ascending=False, na_position='first').reset_index(drop=True)
        df_styled = pivot_sales[['supplierArticle', 'qty', 'PriceWithDiscount']].head(50).style.set_precision(0).background_gradient(
            axis=0, gmap=pivot_sales['PriceWithDiscount'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Продажи за вчера, общая сумма -" 
                                            f" {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_sales['qty'])}.\nКартинкой выводится до топ 50 артикулов по сумме продаж.")
            if call.data == 'Продажи_за_вчера':
                await call.answer()
            elif call.data == 'Продажи_за_вчера_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_sales.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
        
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------         

@dp.callback_query_handler(text=["Продажи_за_эту_неделю", 'Продажи_за_эту_неделю_Excel'])
async def sales(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
            'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales['PriceWithDiscount'] = sales['totalPrice'] * ((100 - sales['discountPercent'])/100) * ((100 - sales['promoCodeDiscount'])/100) *((100 - sales['spp'])/100)
        sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        sales['year_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.year
        sales = sales[(sales['week_num'] >= max(sales['week_num'])) & (sales['year_num'] >= max(sales['year_num'])) & (sales['IsStorno'] == 0) & (sales['PriceWithDiscount'] > 0)]
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['supplierArticle'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
            left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index().sort_values(by=['PriceWithDiscount'], 
                                                            ascending=False, na_position='first').reset_index(drop=True)  
        df_styled = pivot_sales[['supplierArticle', 'qty', 'PriceWithDiscount']].head(50).style.set_precision(0).background_gradient(
            axis=0, gmap=pivot_sales['PriceWithDiscount'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Продажи за эту неделю, общая сумма -" 
                                            f" {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_sales['qty'])}.\nКартинкой выводится до топ 50 артикулов по сумме продаж.")
            if call.data == 'Продажи_за_эту_неделю':
                await call.answer()
            elif call.data == 'Продажи_за_эту_неделю_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_sales.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
        
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------                 

@dp.callback_query_handler(text=["Продажи_за_прошлую_неделю", 'Продажи_за_прошлую_неделю_Excel'])
async def sales(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
            'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales['PriceWithDiscount'] = sales['totalPrice'] * ((100 - sales['discountPercent'])/100) * ((100 - sales['promoCodeDiscount'])/100) *((100 - sales['spp'])/100)
        sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        sales['year_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.year
        sales = sales[(sales['week_num'] == max(sales['week_num'])-1) & (sales['year_num'] >= max(sales['year_num'])) & (sales['IsStorno'] == 0) & (sales['PriceWithDiscount'] > 0)]
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['supplierArticle'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
            left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index().sort_values(by=['PriceWithDiscount'], 
                                                            ascending=False, na_position='first').reset_index(drop=True)  
        df_styled = pivot_sales[['supplierArticle', 'qty', 'PriceWithDiscount']].head(50).style.set_precision(0).background_gradient(
            axis=0, gmap=pivot_sales['PriceWithDiscount'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Продажи за прошлую неделю, общая сумма -" 
                                            f" {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_sales['qty'])}.\nКартинкой выводится до топ 50 артикулов по сумме продаж.")
            if call.data == 'Продажи_за_прошлую_неделю':
                await call.answer()
            elif call.data == 'Продажи_за_прошлую_неделю_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_sales.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru') 
            await call.answer()
        
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------                 

@dp.callback_query_handler(text=["Продажи_за_этот_месяц", 'Продажи_за_этот_месяц_Excel'])
async def sales(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
            'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales['PriceWithDiscount'] = sales['totalPrice'] * ((100 - sales['discountPercent'])/100) * ((100 - sales['promoCodeDiscount'])/100) *((100 - sales['spp'])/100)
        sales['month_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.month
        sales['year_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.year
        sales = sales[(sales['month_num'] == max(sales['month_num'])) & (sales['year_num'] >= max(sales['year_num'])) & (sales['IsStorno'] == 0) & (sales['PriceWithDiscount'] > 0)]
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['supplierArticle'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
            left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index().sort_values(by=['PriceWithDiscount'], 
                                                            ascending=False, na_position='first').reset_index(drop=True)  
        df_styled = pivot_sales[['supplierArticle', 'qty', 'PriceWithDiscount']].head(50).style.set_precision(0).background_gradient(
            axis=0, gmap=pivot_sales['PriceWithDiscount'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Продажи за этот месяц, общая сумма -" 
                                            f" {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_sales['qty'])}.\nКартинкой выводится до топ 50 артикулов по сумме продаж.") 
            if call.data == 'Продажи_за_этот_месяц':
                await call.answer()
            elif call.data == 'Продажи_за_этот_месяц_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_sales.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
        
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------                 

@dp.callback_query_handler(text=["Продажи_за_прошлый_месяц", 'Продажи_за_прошлый_месяц_Excel'])
async def sales(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
            'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales['PriceWithDiscount'] = sales['totalPrice'] * ((100 - sales['discountPercent'])/100) * ((100 - sales['promoCodeDiscount'])/100) *((100 - sales['spp'])/100)
        sales['month_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.month
        sales['year_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.year
        sales = sales[(sales['month_num'] == max(sales['month_num'])-1) & (sales['year_num'] >= max(sales['year_num'])) & (sales['IsStorno'] == 0) & (sales['PriceWithDiscount'] > 0)]
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['supplierArticle'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
            left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index().sort_values(by=['PriceWithDiscount'], 
                                                            ascending=False, na_position='first').reset_index(drop=True)  
        df_styled = pivot_sales[['supplierArticle', 'qty', 'PriceWithDiscount']].head(50).style.set_precision(0).background_gradient(
            axis=0, gmap=pivot_sales['PriceWithDiscount'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Продажи за прошлый месяц, общая сумма -" 
                                            f" {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_sales['qty'])}.\nКартинкой выводится до топ 50 артикулов по сумме продаж.") 
            if call.data == 'Продажи_за_прошлый_месяц':
                await call.answer()
            elif call.data == 'Продажи_за_прошлый_месяц_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_sales.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru') 
            await call.answer()
               
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------                 

@dp.callback_query_handler(text=["Продажи_за_последние_30_дней", 'Продажи_за_последние_30_дней_Excel'])
async def sales(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
                'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales['PriceWithDiscount'] = sales['totalPrice']*((100-sales['discountPercent'])/100)*((100-sales['promoCodeDiscount'])/100)*((100-sales['spp'])/100)
        sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        sales = sales[(sales['date'] >= str(current_date - timedelta(30))) & (sales['IsStorno'] == 0) & (sales['PriceWithDiscount'] > 0)]
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['supplierArticle'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['supplierArticle'], aggfunc='sum', margins=False), how="left", 
            left_on='supplierArticle', right_on='supplierArticle').sort_values(by=['PriceWithDiscount'], ascending=False, na_position='first').reset_index().sort_values(by=['PriceWithDiscount'], 
                                                            ascending=False, na_position='first').reset_index(drop=True)
        df_styled = pivot_sales[['supplierArticle', 'qty', 'PriceWithDiscount']].head(50).style.set_precision(0).background_gradient(
            axis=0, gmap=pivot_sales['PriceWithDiscount'], cmap='BuPu')
        dfi.export(df_styled, "D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png")
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mytable.png', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Продажи за последние 30 дней, общая сумма -" 
                                            f" {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} (без учёта СПП),"
                                            f" общее количество - {sum(pivot_sales['qty'])}.\nКартинкой выводится до топ 50 артикулов по сумме продаж.")
            if call.data == 'Продажи_за_последние_30_дней':
                await call.answer()
            elif call.data == 'Продажи_за_последние_30_дней_Excel':
                writer = pd.ExcelWriter('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx' , engine='xlsxwriter')
                pivot_sales.to_excel(writer, sheet_name='Sheet1', index=False)
                writer.save()
                with open("D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\\file.xlsx", 'rb') as file:
                    await call.message.answer_document(file)
                await call.answer()
    except:
        if sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru') 
            await call.answer()
              
##########   ЗАКАЗЫ ПРОДАЖИ ГРАФИК   #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!    
##########   ЗАКАЗЫ ПРОДАЖИ ГРАФИК   #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!        
##########   ЗАКАЗЫ ПРОДАЖИ ГРАФИК   #########################################################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 

@dp.message_handler(commands="График_заказов_и_продаж")
async def InlKB(message: types.Message):
    keyboard = InlineKeyboardMarkup()
    keyboard.add(types.InlineKeyboardButton(text="График за последние 7 дней", callback_data="За_последние_7_дней"))
    keyboard.add(types.InlineKeyboardButton(text="График за текущий месяц", callback_data="За_текущий_месяц"))
    keyboard.add(types.InlineKeyboardButton(text="График за последние 30 дней", callback_data="За_последние_30_дней"))
    keyboard.add(types.InlineKeyboardButton(text="График за 3 месяца", callback_data="За_3_месяца"))
    keyboard.add(types.InlineKeyboardButton(text="График за последние 90 дней", callback_data="За_последние_90_дней"))
    keyboard.add(types.InlineKeyboardButton(text="График за 6 месяцев", callback_data="За_6_месяцев"))
    keyboard.add(types.InlineKeyboardButton(text="График за последние 180 дней", callback_data="За_последние_180_дней"))
    keyboard.add(types.InlineKeyboardButton(text="График за текущий год", callback_data="За_текущий_год"))
    keyboard.add(types.InlineKeyboardButton(text="График за последние 365 дней", callback_data="За_последние_365_дней"))
    keyboard.add(types.InlineKeyboardButton(text="График за всё время", callback_data="За_всё_время"))    
    await message.answer("Выберите период, заказы по которому вы хотите узнать", reply_markup=keyboard) 

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
@dp.callback_query_handler(text="За_последние_7_дней")
async def graphics(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders = orders[(orders['date'] >= str(current_date - timedelta(6)))]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders['date'] = pd.to_datetime(orders['date'], errors='coerce').dt.strftime('%Y-%m-%d')
        #orders['day_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.day
        #orders['week_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.isocalendar().week
        #orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
        #orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
        #orders['yearweek_num'] = (orders['year_num'].astype(str) + orders['week_num'].astype(str)).astype(int)
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['date'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders, 
            values='PriceWithDiscount', index=['date'], aggfunc='sum', margins=False), how="left", 
            left_on='date', right_on='date').reset_index().sort_values(by=['date'], ascending=True, na_position='first').reset_index(drop=True)

        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
                'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales = sales[(sales['date'] >= str(current_date - timedelta(6)))]
        sales['PriceWithDiscount'] = sales['totalPrice']*((100-sales['discountPercent'])/100)*((100-sales['promoCodeDiscount'])/100)*((100-sales['spp'])/100)
        sales['date'] = pd.to_datetime(sales['date']).dt.strftime('%Y-%m-%d')
        #sales['day_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.day
        #sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        #sales['month_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.month
        #sales['year_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.year
        #sales['yearweek_num'] = (sales['year_num'].astype(str) + sales['week_num'].astype(str)).astype(int)
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['date'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['date'], aggfunc='sum', margins=False), how="left", 
            left_on='date', right_on='date').reset_index().sort_values(by=['date'], ascending=True, na_position='first').reset_index(drop=True)
        
        plt.figure(figsize=(17, 9))
        plt.title('Статистика заказов (с СПП) и продаж (без СПП) по дням за последние 7 дней', fontsize = 25)
        plt.ylabel('Рубли', fontsize = 17)
        plt.xlabel('Дата', fontsize = 17)
        plt.xticks(fontsize = 15)
        plt.yticks(fontsize = 15)
        plot(pivot_orders['date'], pivot_orders['PriceWithDiscount'], color='deeppink', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Заказы')
        plot(pivot_sales['date'], pivot_sales['PriceWithDiscount'], color='navy', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Продажи')
        plt.legend(loc="upper left", fontsize = 15)
        plt.savefig('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', bbox_inches='tight')
        plt.close()    
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказано за последние 7 дней {sum(pivot_orders['qty'])} шт на {int(round(sum(pivot_orders['PriceWithDiscount']), 0))} рублей.\n"
                                            f"Продано за последние 7 дней {sum(pivot_sales['qty'])} шт на {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} рублей.") 
            await call.answer()
    except:
        if orders.shape[0] == 0 | sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов или продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
        
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
@dp.callback_query_handler(text="За_текущий_месяц")
async def graphics(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders['date'] = pd.to_datetime(orders['date'], errors='coerce').dt.strftime('%Y-%m-%d')
        #orders['day_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.day
        #orders['week_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.isocalendar().week
        orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
        orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
        #orders['yearweek_num'] = (orders['year_num'].astype(str) + orders['week_num'].astype(str)).astype(int)
        orders = orders[(orders['month_num'] >= max(orders['month_num'])) & (orders['year_num'] >= max(orders['year_num']))]
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['date'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders, 
            values='PriceWithDiscount', index=['date'], aggfunc='sum', margins=False), how="left", 
            left_on='date', right_on='date').reset_index().sort_values(by=['date'], ascending=True, na_position='first').reset_index(drop=True)

        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
                'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales['PriceWithDiscount'] = sales['totalPrice']*((100-sales['discountPercent'])/100)*((100-sales['promoCodeDiscount'])/100)*((100-sales['spp'])/100)
        sales['date'] = pd.to_datetime(sales['date']).dt.strftime('%Y-%m-%d')
        #sales['day_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.day
        #sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        sales['month_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.month
        sales['year_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.year
        #sales['yearweek_num'] = (sales['year_num'].astype(str) + sales['week_num'].astype(str)).astype(int)
        sales = sales[(sales['month_num'] >= max(sales['month_num'])) & (sales['year_num'] >= max(sales['year_num']))]
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['date'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['date'], aggfunc='sum', margins=False), how="left", 
            left_on='date', right_on='date').reset_index().sort_values(by=['date'], ascending=True, na_position='first').reset_index(drop=True)
        
        plt.figure(figsize=(17, 9))
        plt.title('Статистика заказов (с СПП) и продаж (без СПП) по дням за текущий месяц', fontsize = 25)
        plt.ylabel('Рубли', fontsize = 17)
        plt.xlabel('Дата', fontsize = 17)
        plt.xticks(fontsize = 15)
        plt.yticks(fontsize = 15)
        plot(pd.to_datetime(pivot_orders['date'], errors='coerce').dt.day, pivot_orders['PriceWithDiscount'], color='deeppink', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Заказы')
        plot(pd.to_datetime(pivot_sales['date'], errors='coerce').dt.day, pivot_sales['PriceWithDiscount'], color='navy', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Продажи')
        plt.legend(loc="upper left", fontsize = 15)
        plt.savefig('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', bbox_inches='tight')
        plt.close()      
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказано за текущий месяц {sum(pivot_orders['qty'])} шт на {int(round(sum(pivot_orders['PriceWithDiscount']), 0))} рублей.\n"
                                            f"Продано за текущий месяц {sum(pivot_sales['qty'])} шт на {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} рублей.") 
            await call.answer()
    except:
        if orders.shape[0] == 0 | sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов или продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
        
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
@dp.callback_query_handler(text="За_последние_30_дней")
async def graphics(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders = orders[(orders['date'] >= str(current_date - timedelta(29)))]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders['date'] = pd.to_datetime(orders['date'], errors='coerce').dt.strftime('%Y-%m-%d')
        #orders['day_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.day
        #orders['week_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.isocalendar().week
        #orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
        #orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
        #orders['yearweek_num'] = (orders['year_num'].astype(str) + orders['week_num'].astype(str)).astype(int)
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['date'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders, 
            values='PriceWithDiscount', index=['date'], aggfunc='sum', margins=False), how="left", 
            left_on='date', right_on='date').reset_index().sort_values(by=['date'], ascending=True, na_position='first').reset_index(drop=True)

        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
                'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales = sales[(sales['date'] >= str(current_date - timedelta(29)))]
        sales['PriceWithDiscount'] = sales['totalPrice']*((100-sales['discountPercent'])/100)*((100-sales['promoCodeDiscount'])/100)*((100-sales['spp'])/100)
        sales['date'] = pd.to_datetime(sales['date']).dt.strftime('%Y-%m-%d')
        #sales['day_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.day
        #sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        #sales['month_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.month
        #sales['year_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.year
        #sales['yearweek_num'] = (sales['year_num'].astype(str) + sales['week_num'].astype(str)).astype(int)
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['date'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['date'], aggfunc='sum', margins=False), how="left", 
            left_on='date', right_on='date').reset_index().sort_values(by=['date'], ascending=True, na_position='first').reset_index(drop=True)
        
        plt.figure(figsize=(17, 9))
        plt.title('Статистика заказов (с СПП) и продаж (без СПП) по дням за последние 30 дней', fontsize = 25)
        plt.ylabel('Рубли', fontsize = 17)
        plt.xlabel('Дата', fontsize = 17)
        plt.xticks(fontsize = 15)
        plt.yticks(fontsize = 15)
        plot(pivot_orders['date'], pivot_orders['PriceWithDiscount'], color='deeppink', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Заказы')
        plot(pivot_sales['date'], pivot_sales['PriceWithDiscount'], color='navy', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Продажи')
        plt.legend(loc="upper left", fontsize = 15)
        plt.savefig('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', bbox_inches='tight')
        plt.close()      
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказано за последние 30 дней {sum(pivot_orders['qty'])} шт на {int(round(sum(pivot_orders['PriceWithDiscount']), 0))} рублей.\n"
                                            f"Продано за последние 30 дней {sum(pivot_sales['qty'])} шт на {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} рублей.") 
            await call.answer()
    except:
        if orders.shape[0] == 0 | sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов или продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
@dp.callback_query_handler(text="За_последние_90_дней")
async def graphics(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders = orders[(orders['date'] >= str(current_date - timedelta(89)))]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders['date'] = pd.to_datetime(orders['date'], errors='coerce').dt.strftime('%Y-%m-%d')
        #orders['day_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.day
        orders['week_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.isocalendar().week
        #orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
        orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
        orders['yearweek_num'] = (orders['year_num'].astype(str) + orders['week_num'].astype(str)).astype(int)
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['yearweek_num'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders, 
            values='PriceWithDiscount', index=['yearweek_num'], aggfunc='sum', margins=False), how="left", 
            left_on='yearweek_num', right_on='yearweek_num').reset_index().sort_values(by=['yearweek_num'], ascending=True, na_position='first').reset_index(drop=True)

        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
                'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales = sales[(sales['date'] >= str(current_date - timedelta(89)))]
        sales['PriceWithDiscount'] = sales['totalPrice']*((100-sales['discountPercent'])/100)*((100-sales['promoCodeDiscount'])/100)*((100-sales['spp'])/100)
        sales['date'] = pd.to_datetime(sales['date']).dt.strftime('%Y-%m-%d')
        #sales['day_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.day
        sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        #sales['month_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.month
        sales['year_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.year
        sales['yearweek_num'] = (sales['year_num'].astype(str) + sales['week_num'].astype(str)).astype(int)
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['yearweek_num'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['yearweek_num'], aggfunc='sum', margins=False), how="left", 
            left_on='yearweek_num', right_on='yearweek_num').reset_index().sort_values(by=['yearweek_num'], ascending=True, na_position='first').reset_index(drop=True)
        
        plt.figure(figsize=(17, 9))
        plt.title('Статистика заказов (с СПП) и продаж (без СПП) по дням за последние 90 дней', fontsize = 25)
        plt.ylabel('Рубли', fontsize = 17)
        plt.xlabel('Номер недели в году в формате год_неделя', fontsize = 17)
        plt.xticks(fontsize = 15)
        plt.yticks(fontsize = 15)
        plot(pivot_orders['yearweek_num'], pivot_orders['PriceWithDiscount'], color='deeppink', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Заказы')
        plot(pivot_sales['yearweek_num'], pivot_sales['PriceWithDiscount'], color='navy', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Продажи')
        plt.legend(loc="upper left", fontsize = 15)
        plt.savefig('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', bbox_inches='tight')
        plt.close()      
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказано за последние 90 дней {sum(pivot_orders['qty'])} шт на {int(round(sum(pivot_orders['PriceWithDiscount']), 0))} рублей.\n"
                                            f"Продано за последние 90 дней {sum(pivot_sales['qty'])} шт на {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} рублей.")
            await call.answer()       
    except:
        if orders.shape[0] == 0 | sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов или продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
@dp.callback_query_handler(text="За_последние_180_дней")
async def graphics(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders = orders[(orders['date'] >= str(current_date - timedelta(179)))]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders['date'] = pd.to_datetime(orders['date'], errors='coerce').dt.strftime('%Y-%m-%d')
        #orders['day_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.day
        orders['week_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.isocalendar().week
        #orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
        orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
        orders['yearweek_num'] = (orders['year_num'].astype(str) + orders['week_num'].astype(str)).astype(int)
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['yearweek_num'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders, 
            values='PriceWithDiscount', index=['yearweek_num'], aggfunc='sum', margins=False), how="left", 
            left_on='yearweek_num', right_on='yearweek_num').reset_index().sort_values(by=['yearweek_num'], ascending=True, na_position='first').reset_index(drop=True)

        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
                'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales = sales[(sales['date'] >= str(current_date - timedelta(179)))]
        sales['PriceWithDiscount'] = sales['totalPrice']*((100-sales['discountPercent'])/100)*((100-sales['promoCodeDiscount'])/100)*((100-sales['spp'])/100)
        sales['date'] = pd.to_datetime(sales['date']).dt.strftime('%Y-%m-%d')
        #sales['day_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.day
        sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        #sales['month_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.month
        sales['year_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.year
        sales['yearweek_num'] = (sales['year_num'].astype(str) + sales['week_num'].astype(str)).astype(int)
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['yearweek_num'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['yearweek_num'], aggfunc='sum', margins=False), how="left", 
            left_on='yearweek_num', right_on='yearweek_num').reset_index().sort_values(by=['yearweek_num'], ascending=True, na_position='first').reset_index(drop=True)
        
        plt.figure(figsize=(17, 9))
        plt.title('Статистика заказов (с СПП) и продаж (без СПП) по дням за последние 180 дней', fontsize = 25)
        plt.ylabel('Рубли', fontsize = 17)
        plt.xlabel('Номер недели в году в формате год_неделя', fontsize = 17)
        plt.xticks(fontsize = 15)
        plt.yticks(fontsize = 15)
        plot(pivot_orders['yearweek_num'], pivot_orders['PriceWithDiscount'], color='deeppink', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Заказы')
        plot(pivot_sales['yearweek_num'], pivot_sales['PriceWithDiscount'], color='navy', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Продажи')
        plt.legend(loc="upper left", fontsize = 15)
        plt.savefig('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', bbox_inches='tight')
        plt.close()      
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказано за последние 180 дней {sum(pivot_orders['qty'])} шт на {int(round(sum(pivot_orders['PriceWithDiscount']), 0))} рублей.\n"
                                            f"Продано за последние 180 дней {sum(pivot_sales['qty'])} шт на {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} рублей.") 
            await call.answer()       
    except:
        if orders.shape[0] == 0 | sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов или продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
        
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
@dp.callback_query_handler(text="За_последние_365_дней")
async def graphics(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders = orders[(orders['date'] >= str(current_date - timedelta(364)))]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders['date'] = pd.to_datetime(orders['date'], errors='coerce').dt.strftime('%Y-%m-%d')
        #orders['day_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.day
        orders['week_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.isocalendar().week
        #orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
        orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
        orders['yearweek_num'] = (orders['year_num'].astype(str) + orders['week_num'].astype(str)).astype(int)
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['yearweek_num'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders, 
            values='PriceWithDiscount', index=['yearweek_num'], aggfunc='sum', margins=False), how="left", 
            left_on='yearweek_num', right_on='yearweek_num').reset_index().sort_values(by=['yearweek_num'], ascending=True, na_position='first').reset_index(drop=True)

        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
                'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales = sales[(sales['date'] >= str(current_date - timedelta(364)))]
        sales['PriceWithDiscount'] = sales['totalPrice']*((100-sales['discountPercent'])/100)*((100-sales['promoCodeDiscount'])/100)*((100-sales['spp'])/100)
        sales['date'] = pd.to_datetime(sales['date']).dt.strftime('%Y-%m-%d')
        #sales['day_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.day
        sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        #sales['month_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.month
        sales['year_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.year
        sales['yearweek_num'] = (sales['year_num'].astype(str) + sales['week_num'].astype(str)).astype(int)
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['yearweek_num'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['yearweek_num'], aggfunc='sum', margins=False), how="left", 
            left_on='yearweek_num', right_on='yearweek_num').reset_index().sort_values(by=['yearweek_num'], ascending=True, na_position='first').reset_index(drop=True)
        
        plt.figure(figsize=(17, 9))
        plt.title('Статистика заказов (с СПП) и продаж (без СПП) по дням за последние 365 дней', fontsize = 25)
        plt.ylabel('Рубли', fontsize = 17)
        plt.xlabel('Номер недели в году в формате год_неделя', fontsize = 17)
        plt.xticks(fontsize = 15)
        plt.yticks(fontsize = 15)
        plot(pivot_orders['yearweek_num'], pivot_orders['PriceWithDiscount'], color='deeppink', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Заказы')
        plot(pivot_sales['yearweek_num'], pivot_sales['PriceWithDiscount'], color='navy', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Продажи')
        plt.legend(loc="upper left", fontsize = 15)
        plt.savefig('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', bbox_inches='tight')
        plt.close()      
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказано за последние 365 дней {sum(pivot_orders['qty'])} шт на {int(round(sum(pivot_orders['PriceWithDiscount']), 0))} рублей.\n"
                                            f"Продано за последние 365 дней {sum(pivot_sales['qty'])} шт на {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} рублей.")
            await call.answer()      
    except:
        if orders.shape[0] == 0 | sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов или продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
                       
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------        
        
@dp.callback_query_handler(text="За_всё_время")
async def graphics(call: types.CallbackQuery): 
    try:
        current_date = date.today()
        response_orders = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/orders?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'warehouseName', 'oblast', 'subject', 'category', 'cancel_dt']
        orders = pd.DataFrame(json.loads(response_orders.content.decode('utf-8')))[cols]
        orders['PriceWithDiscount'] = orders['totalPrice'] * (1 - orders['discountPercent'] / 100)
        orders['date'] = pd.to_datetime(orders['date'], errors='coerce').dt.strftime('%Y-%m-%d')
        #orders['day_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.day
        orders['week_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.isocalendar().week
        #orders['month_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.month
        orders['year_num'] = pd.to_datetime(orders['date'], errors='coerce').dt.year
        orders['yearweek_num'] = (orders['year_num'].astype(str) + orders['week_num'].astype(str)).astype(int)
        pivot_orders = pd.merge(pd.pivot_table(orders, values='PriceWithDiscount', index=['yearweek_num'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(orders, 
            values='PriceWithDiscount', index=['yearweek_num'], aggfunc='sum', margins=False), how="left", 
            left_on='yearweek_num', right_on='yearweek_num').reset_index().sort_values(by=['yearweek_num'], ascending=True, na_position='first').reset_index(drop=True)

        response_sales = requests.get('https://statistics-api.wildberries.ru/api/v1/supplier/sales?dateFrom=2017-03-25T21%3A00%3A00.000Z&flag=0',
            headers={'Authorization': f'{df_user_api_excel[df_user_api_excel["user_id"] == str(call.from_user.id)].reset_index(drop=True)["user_api"].iloc[-1]}'})
        cols = ['date', 'supplierArticle', 'totalPrice', 'discountPercent', 'promoCodeDiscount', 'warehouseName', 'subject', 
                'category', 'regionName', 'spp', 'forPay', 'finishedPrice', 'priceWithDisc', 'IsStorno']
        sales = pd.DataFrame(json.loads(response_sales.content.decode('utf-8')))[cols]
        sales['PriceWithDiscount'] = sales['totalPrice']*((100-sales['discountPercent'])/100)*((100-sales['promoCodeDiscount'])/100)*((100-sales['spp'])/100)
        sales['date'] = pd.to_datetime(sales['date']).dt.strftime('%Y-%m-%d')
        #sales['day_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.day
        sales['week_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.isocalendar().week
        #sales['month_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.month
        sales['year_num'] = pd.to_datetime(sales['date'], errors='coerce').dt.year
        sales['yearweek_num'] = (sales['year_num'].astype(str) + sales['week_num'].astype(str)).astype(int)
        pivot_sales = pd.merge(pd.pivot_table(sales, values='PriceWithDiscount', index=['yearweek_num'], 
            aggfunc='count', margins=False).rename(columns={'PriceWithDiscount': 'qty'}), pd.pivot_table(sales, 
            values='PriceWithDiscount', index=['yearweek_num'], aggfunc='sum', margins=False), how="left", 
            left_on='yearweek_num', right_on='yearweek_num').reset_index().sort_values(by=['yearweek_num'], ascending=True, na_position='first').reset_index(drop=True)
        
        plt.figure(figsize=(17, 9))
        plt.title('Статистика заказов (с СПП) и продаж (без СПП) по неделям за всё время', fontsize = 25)
        plt.ylabel('Рубли', fontsize = 17)
        plt.xlabel('Номер недели в году в формате год_неделя', fontsize = 17)
        plt.xticks(fontsize = 15)
        plt.yticks(fontsize = 15)
        plot(pivot_orders['yearweek_num'], pivot_orders['PriceWithDiscount'], color='deeppink', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Заказы')
        plot(pivot_sales['yearweek_num'], pivot_sales['PriceWithDiscount'], color='navy', marker='o', linestyle='dashed',
             linewidth=2, markersize=12, label='Продажи')
        plt.legend(loc="upper left", fontsize = 15)
        plt.savefig('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', bbox_inches='tight')
        plt.close()
        with open('D:\грузи сюда\Всё по WB\ВСЁ НАШЕЕЕЕЕЕ\mygraph.jpg', 'rb') as photo:
            await call.message.answer_photo(photo = photo, caption=f"Заказано за всё время {sum(pivot_orders['qty'])} шт на {int(round(sum(pivot_orders['PriceWithDiscount']), 0))} рублей.\n"
                                            f"Продано за всё время {sum(pivot_sales['qty'])} шт на {int(round(sum(pivot_sales['PriceWithDiscount']), 0))} рублей.")
            await call.answer()    
    except:
        if orders.shape[0] == 0 | sales.shape[0] == 0:
            await call.message.answer(f'За этот период ещё не было заказов или продаж \U0001F9D0\U0001F9D0\U0001F9D0')
            await call.answer()
        else:
            await call.message.answer('\U0001FAE0 Что-то пошло не так \U0001FAE0 \nНапиши создателю бота @dimi3_tru')
            await call.answer()
        
        
        
        
        
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!        
        
if __name__ == '__main__':
    executor.start_polling(dp, skip_updates=True)

Task exception was never retrieved
future: <Task finished name='Task-361' coro=<Executor._startup_polling() done, defined at D:\ANACONDA\lib\site-packages\aiogram\utils\executor.py:371> exception=NetworkError('Aiohttp client throws an error: ClientOSError: [Errno 1] [SSL: APPLICATION_DATA_AFTER_CLOSE_NOTIFY] application data after close notify (_ssl.c:2756)')>
Traceback (most recent call last):
  File "D:\ANACONDA\lib\site-packages\aiogram\bot\api.py", line 139, in make_request
    async with session.post(url, data=req, **kwargs) as response:
  File "D:\ANACONDA\lib\site-packages\aiohttp\client.py", line 1138, in __aenter__
    self._resp = await self._coro
  File "D:\ANACONDA\lib\site-packages\aiohttp\client.py", line 559, in _request
    await resp.start(conn)
  File "D:\ANACONDA\lib\site-packages\aiohttp\client_reqrep.py", line 898, in start
    message, payload = await protocol.read()  # type: ignore[union-attr]
  File "D:\ANACONDA\lib\site-packages\aiohttp\streams.py", line 616, i

In [ ]:
@dp.message_handler(commands="Продажи_по_товарам")
async def InlKB(message: types.Message):
    keyboard = types.InlineKeyboardMarkup()
    keyboard.add(types.InlineKeyboardButton(text="Продажи за сегодня", callback_data="Продажи_за_сегодня"))
    keyboard.add(types.InlineKeyboardButton(text="Продажи за эту неделю", callback_data="Продажи_за_эту_неделю"))
    keyboard.add(types.InlineKeyboardButton(text="Продажи за прошлую неделю", callback_data="Продажи_за_прошлую_неделю"))
    keyboard.add(types.InlineKeyboardButton(text="Продажи за этот месяц", callback_data="Продажи_за_этот_месяц"))
    keyboard.add(types.InlineKeyboardButton(text="Продажи за прошлый месяц", callback_data="Продажи_за_прошлый_месяц"))
    keyboard.add(types.InlineKeyboardButton(text="Продажи за последние 30 дней", callback_data="Продажи_за_последние_30_дней"))
    await message.answer("Выберите период, Продажи по которому вы хотите узнать", reply_markup=keyboard) 